---
6 Natural Language Processing (NLP)
---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SMC-AAU-CPH/ML-For-Beginners/blob/main/6-NLP/images/6-NLP.ipynb)


In [1]:
try:
  import google.colab
  IN_COLAB = True
  # !mkdir -p data
  # !wget https://raw.githubusercontent.com/SMC-AAU-CPH/med4-ap-jupyter/main/lecture7_Fourer_Transfom/data/trumpet.wav -P data
  # !curl https://raw.githubusercontent.com/cs109/2015lab8/master/data/pride_and_prejudice.txt -o pride.txt
  !pip install pandas==1.5.3 pycaret==2.0.0 textblob
except:
  IN_COLAB = False

In [2]:
# check version
from pycaret.utils import version
version()


2.0


# 6.1 Introduction
Where we learn about Elisa, Turing Test, and Computational Lingustics, then build a (random) conversational bot.

## 6.1.1 The plan

Your steps when building a conversational bot:

1. Print instructions advising the user how to interact with the bot
2. Start a loop
    * Accept user input
    * If user has asked to exit, then exit
    * Process user input and determine response (in this case, the response is a random choice from a list of possible generic responses)
    * Print response
3. loop back to step 2

In [3]:
import random

# This list contains the random responses (you can add your own or translate them into your own language too)
random_responses = ["That is quite interesting, please tell me more.",
                    "I see. Do go on.",
                    "Why do you say that?",
                    "Funny weather we've been having, isn't it?",
                    "Let's change the subject.",
                    "Did you catch the game last night?"]

print("Hello, I am Marvin, the simple robot.")
print("You can end this conversation at any time by typing 'bye'")
print("After typing each answer, press 'enter'")
print("How are you today?")

while True:
    # wait for the user to enter some text
    user_input = input("> ")
    if user_input.lower() == "bye":
        # if they typed in 'bye' (or even BYE, ByE, byE etc.), break out of the loop
        break
    else:
        response = random.choices(random_responses)[0]
    print(response)

print("It was nice talking to you, goodbye!")

Hello, I am Marvin, the simple robot.
You can end this conversation at any time by typing 'bye'
After typing each answer, press 'enter'
How are you today?
> bye
It was nice talking to you, goodbye!


✅ Stop and consider

* Do you think the random responses would 'trick' someone into thinking that the bot actually understood them?
* What features would the bot need to be more effective?
* If a bot could really 'understand' the meaning of a sentence, would it need to 'remember' the meaning of previous sentences in a conversation too?


# 6.2 [Common NLP tasks](https://paperswithcode.com/area/natural-language-processing) and techniques
Where we learn about **tokenization** (splitting text into *tokens*), Embeddings (convert text to numerical data so that similar / related words cluster), Parsing & Part of speech tagging (e.g., as a noun, verb, or adjective), word and phrase frequencies, n-grams (A text can be split into sequences of words of a set length $n$), Noun phrase Extraction, [Sentiment analysis](https://paperswithcode.com/task/sentiment-analysis), inflection, and lemmatization (rooting of words). There are useful databases like WordNet, [GLUE and SST](https://paperswithcode.com/datasets?task=sentiment-analysis) and frameworks from classical NLP (e.g., TextBlob) or Deep Learning (e.g., [Transormers](https://huggingface.co/learn/nlp-course)).

💡Take a look at a [deep learning embeddings on TensorFlow (and recall PCA and T-SNE)](https://projector.tensorflow.org)



### 6.2.1 Exercise  - using `TextBlob` library

Let's use a library called TextBlob as it contains helpful APIs for tackling these types of tasks. TextBlob "stands on the giant shoulders of [NLTK](https://nltk.org) and [pattern](https://github.com/clips/pattern), and plays nicely with both." It has a considerable amount of ML embedded in its API.

> Note: A useful [Quick Start](https://textblob.readthedocs.io/en/dev/quickstart.html#quickstart) guide is available for TextBlob that is recommended for experienced Python developers

When attempting to identify *noun phrases*, TextBlob offers several options of extractors to find noun phrases.

1. Take a look at `ConllExtractor`.

   ```python
   from textblob import TextBlob
   from textblob.np_extractors import ConllExtractor
   # import and create a Conll extractor to use later
   extractor = ConllExtractor()

   # later when you need a noun phrase extractor:
   user_input = input("> ")
   user_input_blob = TextBlob(user_input, np_extractor=extractor)  # note non-default extractor specified
   np = user_input_blob.noun_phrases                                  
   ```
   > What's going on here? [ConllExtractor](https://textblob.readthedocs.io/en/dev/api_reference.html?highlight=Conll#textblob.en.np_extractors.ConllExtractor) is "A noun phrase extractor that uses chunk parsing trained with the ConLL-2000 training corpus." ConLL-2000 refers to the 2000 Conference on Computational Natural Language Learning. Each year the conference hosted a workshop to tackle a thorny NLP problem, and in 2000 it was noun chunking. A model was trained on the Wall Street Journal, with "sections 15-18 as training data (211727 tokens) and section 20 as test data (47377 tokens)". You can look at the procedures used [here](https://www.clips.uantwerpen.be/conll2000/chunking/) and the [results](https://ifarm.nl/erikt/research/np-chunking.html).
   >

### 6.2.2 Challenge - improving our bot with NLP

In 6.1 we built a very simple Q&A bot. Now, we'll make Marvin a bit more sympathetic by analyzing your input for sentiment and printing out a response to match the sentiment. We'll also need to identify a `noun_phrase` and ask about it.

Our steps when building a better conversational bot:

1. Print instructions advising the user how to interact with the bot
2. Start loop
   1. Accept user input
   2. If user has asked to exit, then exit
   3. Process user input and determine appropriate sentiment response
   4. If a noun phrase is detected in the sentiment, pluralize it and ask for more input on that topic
   5. Print response
3. loop back to step 2

In [4]:
import random
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor
import nltk
nltk.download('punkt')
extractor = ConllExtractor()


nltk.download('conll2000')

print("Hello, I am Marvin, the simple robot.")
print("You can end this conversation at any time by typing 'bye'")
print("After typing each answer, press 'enter'")
print("How are you today?")

while True:
        # wait for the user to enter some text
        user_input = input("> ")

        if user_input.lower() == "bye":
            # if they typed in 'bye' (or even BYE, ByE, byE etc.), break out of the loop
            break
        else:
            # Create a TextBlob based on the user input. Then extract the noun phrases
            user_input_blob = TextBlob(user_input, np_extractor=extractor)
            np = user_input_blob.noun_phrases
            response = ""
            if user_input_blob.polarity <= -0.5:
                response = "Oh dear, that sounds bad. "
            elif user_input_blob.polarity <= 0:
                response = "Hmm, that's not great. "
            elif user_input_blob.polarity <= 0.5:
                response = "Well, that sounds positive. "
            elif user_input_blob.polarity <= 1:
                response = "Wow, that sounds great. "

            if len(np) != 0:
                # There was at least one noun phrase detected, so ask about that and pluralise it
                # e.g. cat -> cats or mouse -> mice
                response = response + "Can you tell me more about " + np[0].pluralize() + "?"
            else:
                response = response + "Can you tell me more?"
            print(response)

print("It was nice talking to you, goodbye!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


Hello, I am Marvin, the simple robot.
You can end this conversation at any time by typing 'bye'
After typing each answer, press 'enter'
How are you today?
> bye
It was nice talking to you, goodbye!


# 6.3 Translation and sentiment analysis with ML

### 6.3.1 Translation

In [5]:
from textblob import TextBlob

blob = TextBlob(
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife!"
)
blob.translate(from_lang="en",to="da")

TextBlob("Det er en sandhed, der universelt er anerkendt, at en enkelt mand, der er i besiddelse af en lykke, skal være i mangel på en kone!")

> What's going on here? and why is TextBlob so good at translation? Well, behind the scenes, it's using Google translate, a sophisticated AI able to parse millions of phrases to predict the best strings for the task at hand. There's nothing manual going on here and you need an internet connection to use `blob.translate`.

✅ Try some more sentences. Which is better, ML or human translation? In which cases?

### 6.3.2 Sentiment
Sentiment is measured in with a *polarity* of -1 to 1, meaning -1 is the most negative sentiment, and 1 is the most positive.

Sentiment is also measured with an 0 - 1 score for objectivity (0) and subjectivity (1).

In [6]:
from textblob import TextBlob

quote1 = """It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife."""

quote2 = """Darcy, as well as Elizabeth, really loved them; and they were both ever sensible of the warmest gratitude towards the persons who, by bringing her into Derbyshire, had been the means of uniting them."""

sentiment1 = TextBlob(quote1).sentiment
sentiment2 = TextBlob(quote2).sentiment

print(quote1 + " has a sentiment of " + str(sentiment1))
print(quote2 + " has a sentiment of " + str(sentiment2))

It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. has a sentiment of Sentiment(polarity=0.20952380952380953, subjectivity=0.27142857142857146)
Darcy, as well as Elizabeth, really loved them; and they were both ever sensible of the warmest gratitude towards the persons who, by bringing her into Derbyshire, had been the means of uniting them. has a sentiment of Sentiment(polarity=0.7, subjectivity=0.8)


#### Challenge - check sentiment polarity
Our task is to determine, using sentiment polarity, if *Pride and Prejudice* has more absolutely positive sentences than absolutely negative ones. For this task, you may assume that a polarity score of 1 or -1 is absolutely positive or negative respectively.

**Steps:**

1. ~~Download a [copy of Pride and Prejudice](https://www.gutenberg.org/files/1342/1342-h/1342-h.htm) from Project Gutenberg as a .txt file. Remove the metadata at the start and end of the file, leaving only the original text~~ Cumhur used `curl https://raw.githubusercontent.com/cs109/2015lab8/master/data/pride_and_prejudice.txt -o pride.txt` and did not clean up (as metadata is neutral).
2. Open the file in Python and extract the contents as a string
3. Create a TextBlob using the book string
4. Analyse each sentence in the book in a loop
   1. If the polarity is 1 or -1 store the sentence in an array or list of positive or negative messages
5. At the end, print out all the positive sentences and negative sentences (separately) and the number of each.

In [7]:
from textblob import TextBlob


In [8]:
# You should download the book text, clean it, and import it here
with open("pride.txt", encoding="utf8") as f:
    file_contents = f.read()


In [9]:
book_pride = TextBlob(file_contents)
positive_sentiment_sentences = []
negative_sentiment_sentences = []

In [10]:
for sentence in book_pride.sentences:
    if sentence.sentiment.polarity == 1:
        positive_sentiment_sentences.append(sentence)
    if sentence.sentiment.polarity == -1:
        negative_sentiment_sentences.append(sentence)


In [11]:
print("The " + str(len(positive_sentiment_sentences)) + " most positive sentences:")
for sentence in positive_sentiment_sentences:
    print("+ " + str(sentence.replace("\n", "").replace("      ", " ")))


The 39 most positive sentences:
+ "What an excellent father you have, girls!"
+ Everybody said how wellshe looked; and Mr. Bingley thought her quite beautiful, and danced withher twice!
+ He walked here, and he walked there, fancying himself so verygreat!
+ Elizabeth assured him that she could suit herself perfectly with thosein the room.
+ What a delightful library you have atPemberley, Mr.
+ Her performance on the pianoforte is exquisite."
+ yes--I understand you perfectly."
+ "I am perfectly convinced by it that Mr. Darcy has no defect.
+ "It _is_ wonderful," replied Wickham, "for almost all his actions maybe traced to pride; and pride had often been his best friend.
+ Family pride, and _filial_ pride--for he is very proud of whathis father was--have done this.
+ _That_ would be the greatest misfortune of all!
+ How wonderfully these sort of things occur!
+ She owedher greatest relief to her friend Miss Lucas, who often joined them, andgood-naturedly engaged Mr. Collins's conversati

In [12]:
print("The " + str(len(negative_sentiment_sentences)) + " most negative sentences:")
for sentence in negative_sentiment_sentences:
    print("- " + str(sentence.replace("\n", "").replace("      ", " ")))

The 18 most negative sentences:
- shocking!"
- Everybody is disgusted with his pride.
- "This is quite shocking!
- What canhave induced him to behave so cruelly?"
- His dispositionmust be dreadful."
- To finda man agreeable whom one is determined to hate!
- "You shall hear then--but prepare yourself for something very dreadful.
- The pause was to Elizabeth's feelingsdreadful.
- "Wickham sovery bad!
- The separationbetween her and her family was rather noisy than pathetic.
- It would be dreadful!
- It is every way horrible!"
- "Oh, yes!--that, that is the worst of all.
- "She is so fond of Mrs. Forster," said she, "it will be quite shockingto send her away!
- It was all over before I arrived; so my curiosity was not sodreadfully racked as _yours_ seems to have been.
- Hecalled it, therefore, his duty to step forward, and endeavour to remedyan evil which had been brought on by himself.
- "Hate you!
- You were disgusted with the women who were always speaking,and looking, and thinking for

✅ Knowledge Check: Consult to the [3-Translation-Sentiment/README.md](../3-Translation-Sentiment/README.md)

# 6.4 Hotel Reviews 1

See [4-Hotel-Reviews-1/README.md](../4-Hotel-Reviews-1/README.md) for details and the questions we ask.

❗️If you don't like downloading from Kaggle or elsewhere as we do, you could try if you have huggingface datasets installed:
 ```python
 from datasets import load_dataset
 dataset = load_dataset("ashraq/hotel-reviews")
 df = dataset.to_pandas()
 df.head() # note the data HF has only 3 columns and already cleaned
```

In [13]:
## EDA
import pandas as pd
import time


In [14]:
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]

In [15]:
# Load the hotel reviews from CSV
print("Loading data file now, this could take a while depending on file size")
start = time.time()
df = pd.read_csv('Hotel_Reviews.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")


Loading data file now, this could take a while depending on file size
Loading took 11.75 seconds


In [16]:
# What shape is the data (rows, columns)?
print("The shape of the data (rows, cols) is " + str(df.shape))


The shape of the data (rows, cols) is (515738, 17)


In [17]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [18]:
# value_counts() creates a Series object that has index and values
#                in this case, the country and the frequency they occur in reviewer nationality
nationality_freq = df["Reviewer_Nationality"].value_counts()


In [19]:
# What reviewer nationality is the most common in the dataset?
print("The highest frequency reviewer nationality is " + str(nationality_freq.index[0]).strip() + " with " + str(nationality_freq[0]) + " reviews.")


The highest frequency reviewer nationality is United Kingdom with 245246 reviews.


In [20]:
# What is the top 10 most common nationalities and their frequencies?
print("The top 10 highest frequency reviewer nationalities are:")
print(nationality_freq[0:10].to_string())


The top 10 highest frequency reviewer nationalities are:
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
 Saudi Arabia                   8951
 Netherlands                    8772
 Switzerland                    8678
 Germany                        7941
 Canada                         7894


In [21]:
# How many unique nationalities are there?
print("There are " + str(nationality_freq.index.size) + " unique nationalities in the dataset")


There are 227 unique nationalities in the dataset


In [22]:
# What was the most frequently reviewed hotel for the top 10 nationalities - print the hotel and number of reviews
for nat in nationality_freq[:10].index:
   # First, extract all the rows that match the criteria into a new dataframe
   nat_df = df[df["Reviewer_Nationality"] == nat]
   # Now get the hotel freq
   freq = nat_df["Hotel_Name"].value_counts()
   print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.")


The most reviewed hotel for United Kingdom was Britannia International Hotel Canary Wharf with 3833 reviews.
The most reviewed hotel for United States of America was Hotel Esther a with 423 reviews.
The most reviewed hotel for Australia was Park Plaza Westminster Bridge London with 167 reviews.
The most reviewed hotel for Ireland was Copthorne Tara Hotel London Kensington with 239 reviews.
The most reviewed hotel for United Arab Emirates was Millennium Hotel London Knightsbridge with 129 reviews.
The most reviewed hotel for Saudi Arabia was The Cumberland A Guoman Hotel with 142 reviews.
The most reviewed hotel for Netherlands was Jaz Amsterdam with 97 reviews.
The most reviewed hotel for Switzerland was Hotel Da Vinci with 97 reviews.
The most reviewed hotel for Germany was Hotel Da Vinci with 86 reviews.
The most reviewed hotel for Canada was St James Court A Taj Hotel London with 61 reviews.


In [23]:
# How many reviews are there per hotel (frequency count of hotel) and do the results match the value in `Total_Number_of_Reviews`?
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)
# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Total_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')
# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"])
# print()
# print(hotel_freq_df.to_string())
# print(str(hotel_freq_df.shape))

In [24]:
# While there is an `Average_Score` for each hotel according to the dataset,
# you can also calculate an average score (getting the average of all reviewer scores in the dataset for each hotel)
# Add a new column to your dataframe with the column header `Calc_Average_Score` that contains that calculated average.
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)
# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)
# Create a df without all the duplicates of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ["Hotel_Name"])
# Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])
print(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])
# Do any hotels have the same (rounded to 1 decimal place) `Average_Score` and `Calc_Average_Score`?


        Average_Score_Difference  Average_Score  Calc_Average_Score  \
495945                      -0.8            7.7                 8.5   
111027                      -0.7            8.8                 9.5   
43688                       -0.7            7.5                 8.2   
178253                      -0.7            7.9                 8.6   
218258                      -0.5            7.0                 7.5   
...                          ...            ...                 ...   
151416                       0.7            7.8                 7.1   
22189                        0.8            7.1                 6.3   
250308                       0.9            8.6                 7.7   
68936                        0.9            6.8                 5.9   
3813                         1.3            7.2                 5.9   

                                               Hotel_Name  
495945                         Best Western Hotel Astoria  
111027  Hotel Stendhal Plac

💡As we proceed to NLP, take this opportunity to read through the '[NLTK book](https://www.nltk.org/book/)' and try out its exercises.

# 6.5-Hotel-Reviews-2: Sentiment Analysis

Now that you have explored the dataset in detail, it's time to filter the columns and then use NLP techniques on the dataset to gain new insights about the hotels.

### Exercise: a bit more data processing

Clean the data just a bit more. Add columns that will be useful later, change the values in other columns, and drop certain columns completely.

In [25]:
import ast

In [26]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria"
    else:
        return row.Hotel_Address


In [27]:
# dropping columns we will not use:
df.drop(["lat", "lng"], axis = 1, inplace=True)

In [28]:
# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)
# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"], axis = 1, inplace=True)

In [29]:
# Replace `Total_Number_of_Reviews` and `Average_Score` with our own calculated values
df.Average_Score = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

In [30]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends,
# Family with young children, Family with older children, With a pet

df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

In [31]:
# No longer need any of these columns
df.drop(["Review_Date", "Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "days_since_review", "Total_Number_of_Reviews_Reviewer_Has_Given"], axis = 1, inplace=True)


In [32]:
# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r'Hotel_Reviews_Filtered.csv', index = False)
end = time.time()
print("Filtering took " + str(round(end - start, 2)) + " seconds")

Saving results to Hotel_Reviews_Filtered.csv
Filtering took 44.54 seconds


### Exercise: load and save the filtered data for sentiment analysis
The code skeleton should like this (assuming) independent execution:
```python
import time
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Load the filtered hotel reviews from CSV
df = pd.read_csv('Hotel_Reviews_Filtered.csv')

# You code will be added here


# Finally remember to save the hotel reviews with new NLP data added
print("Saving results to Hotel_Reviews_NLP.csv")
df.to_csv(r'Hotel_Reviews_NLP.csv', index = False)
```

The actual code is below.

In [33]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [34]:
vader_sentiment = SentimentIntensityAnalyzer()

In [35]:
# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review):
    if review == "No Negative" or review == "No Positive":
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]

In [36]:
# Load the hotel reviews from CSV (already filtered)
df = pd.read_csv("Hotel_Reviews_Filtered.csv")

In [37]:
# Remove stop words - can be slow for a lot of text!
# Ryan Han (ryanxjhan on Kaggle) has a great post measuring performance of different stop words removal approaches
# https://www.kaggle.com/ryanxjhan/fast-stop-words-removal # using the approach that Ryan recommends
start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text


In [38]:
# Remove the stop words from both columns
df.Negative_Review = df.Negative_Review.apply(remove_stopwords)
df.Positive_Review = df.Positive_Review.apply(remove_stopwords)


In [39]:
end = time.time()
print("Removing stop words took " + str(round(end - start, 2)) + " seconds")


Removing stop words took 3.9 seconds


In [40]:
# Add a negative sentiment and positive sentiment column. (This can take a moment, ~1,5 minute)
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews
Calculating sentiment took 193.59 seconds


In [41]:
df = df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(df[["Negative_Review", "Negative_Sentiment"]])
df = df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(df[["Positive_Review", "Positive_Sentiment"]])

                                          Negative_Review  Negative_Sentiment
186584  So bad experience memories I hotel The first n...             -0.9920
129503  First charged twice room booked booking second...             -0.9896
307286  The staff Had bad experience even booking Janu...             -0.9889
201953  Everything DO NOT STAY AT THIS HOTEL I never i...             -0.9886
452092  No WLAN room Incredibly rude restaurant staff ...             -0.9884
...                                                   ...                 ...
138365  Wifi terribly slow I speed test network upload...              0.9938
79215   I find anything hotel first I walked past hote...              0.9938
278506  The property great location There bakery next ...              0.9945
339189  Guys I like hotel I wish return next year Howe...              0.9948
480509  I travel lot far visited countless number hote...              0.9957

[515738 rows x 2 columns]
                                     

In [42]:
# Reorder the columns (This is cosmetic, but to make it easier to explore the data later)
df = df.reindex(["Hotel_Name", "Hotel_Address", "Total_Number_of_Reviews", "Average_Score", "Reviewer_Score", "Negative_Sentiment", "Positive_Sentiment", "Reviewer_Nationality", "Leisure_trip", "Couple", "Solo_traveler", "Business_trip", "Group", "Family_with_young_children", "Family_with_older_children", "With_a_pet", "Negative_Review", "Positive_Review"], axis=1)


In [43]:
print("Saving results to Hotel_Reviews_NLP.csv")
df.to_csv(r"Hotel_Reviews_NLP.csv", index = False)

Saving results to Hotel_Reviews_NLP.csv
